In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy import distance
import geopandas
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from keras.utils import np_utils

import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score

### Function to test new data for Fraud classification

In [4]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score


def predict_fraud(test_file_path, model_file_path):
    
    def calc_distance(lat1, lon1, lat2, lon2):
        R = 6373.0

        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c

        return distance

    # Load test data from file
    test_data = pd.read_csv(test_file_path)

    # Load model from file
    model = Sequential()
    model.load_weights(model_file_path)

    # Perform data preprocessing on test data
    test_data['trans_date_trans_time'] = pd.to_datetime(test_data['trans_date_trans_time'])
    test_data['hour_of_day'] = test_data['trans_date_trans_time'].dt.hour
    test_data['day_of_week'] = test_data['trans_date_trans_time'].dt.dayofweek
    test_data['dob'] = pd.to_datetime(test_data['dob'])
    test_data['age'] = (test_data['trans_date_trans_time'] - test_data['dob']).dt.days // 365
    test_data['distance'] = np.vectorize(calc_distance)(test_data['lat'], test_data['long'], test_data['merch_lat'], test_data['merch_long'])
    test_data.drop(columns=['trans_date_trans_time', 'cc_num', 'merchant', 'first', 'last', 'street', 'city', 'state', 'zip',
                     'lat', 'long', 'merch_lat', 'merch_long', 'job', 'dob', 'trans_num', 'unix_time'], inplace=True)
    test_data = pd.get_dummies(test_data, columns=['category', 'gender'])
    scaler = StandardScaler()
    test_data_scaled = scaler.fit_transform(test_data)

    # Make predictions on test data
    y_pred = model.predict(test_data_scaled)
    y_pred_binary = np.round(y_pred).flatten()

    # Get classification matrix
    cm = confusion_matrix(test_data['is_fraud'], y_pred_binary)
    accuracy = accuracy_score(test_data['is_fraud'], y_pred_binary)

    # Print classification matrix and accuracy
    print("Confusion Matrix:\n", cm)
    print("Accuracy:", accuracy)

    # Return classification matrix and accuracy
    return cm, accuracy


In [ ]:
# Execute function on new data
# cm, accuracy = predict_fraud('test_data.csv', 'model.h5')